In [131]:
import pandas as pd
import numpy as np

In [186]:
clientdf = pd.read_json('original_data/Clients.json')
assistancedf = pd.read_json('original_data/Assistances.json')

clientdf = (clientdf
                .drop(['LastName, FirstName MI', '#inFamily'], axis=1)
                .assign(Gender=np.nan)
                .assign(Race=np.nan)
                .assign(Warning=np.nan)
)


result = pd.merge(clientdf, assistancedf, on='ClientID')
result = (result
            .set_index(['ClientID', 'AssistanceID'])
            .assign(BoyAge=lambda df: df['BoyAge'].astype(str))
            .assign(GirlAge=lambda df: df['GirlAge'].astype(str))
        )
result[['BoyAge', 'GirlAge']] = result[['BoyAge', 'GirlAge']].astype(str)

In [134]:
clientset = set(clientdf['ClientID'])
assistanceset = set(assistancedf['ClientID'])
missing_assistances = assistancedf.loc[assistancedf['ClientID'].isin(assistanceset.difference(clientset))]
missing_clients = clientdf.loc[clientdf['ClientID'].isin(clientset.difference(assistanceset))]

In [135]:
familydf = result.groupby('ClientID').agg(
    BoyAge = pd.NamedAgg(column='BoyAge', aggfunc=lambda x: max(x, key=len)),
    GirlAge = pd.NamedAgg(column='GirlAge', aggfunc=lambda x: max(x, key=len))
)
familydf = (familydf
                .loc[(familydf['BoyAge'] != 'nan') | (familydf['GirlAge'] != 'nan')]
                .assign(NumBoys=lambda df: df['BoyAge'].str.split('-|,').str.len())
                .assign(NumGirls=lambda df: df['GirlAge'].str.split('-|,').str.len())
)
familydf.loc[familydf['BoyAge'] == 'nan', 'NumBoys'] = 0
familydf.loc[familydf['GirlAge'] == 'nan', 'NumGirls'] = 0
familydf['NumKids'] = familydf['NumBoys'] + familydf['NumGirls']

#familydf = familydf[['BoyAge', 'GirlAge', 'NumKids']]
#familydf.reset_index().to_json('Family.json', orient='records')

In [155]:
warningmap = {1135: 'STOLE EILEENS PURSE IN THE CHILDRENS ROOM - BANNED FROM FRANCES HOUSE - PER K.',
  44311: 'emergency banned',
  37842: 'Banned',
  52332: 'BANNED\r\nLaura Young',
  3883: 'BANNED!!!!\r\nHAD TO CALL POLICE ON HER',
  3141: 'Banned until further notice!!!!!!!!!! IF REFUSES TO LEAVE.:CALL 911',
  22413: 'new client\r\nalso uses randy johnson and birthdate of 4-9-63 banned not elibile to return until dec 08',
  22431: 'AGGRESSIVE AND THREATENING - PERMANENTLY BANNED - PER STEPHANIE',
  4959: 'NASTY BELLIGERANT;STARTED FIGHT AND WOULD NOT LEAVE POLICE CAME: BANNED FOR A',
  13608: 'MUST HAVE ID NO EXCEPTIONS!! goes by Gerald Threet\r\n& F Phillip Threet same birthdate check by bday came in may and july under different names! *BANNED FOR 6 MONTHS, SEE DIRECTOR IF HE GIVES ANYONE A BAD TIME.',
  377: 'CANNOT COME IN IF SMELLS OF ALCOHOL!!', 
  20824: 'The police are looking for this guy.  Give him what he wants and notify Alan or Stepahnie.',
  20541: 'also uses last name raymundo (raimundo)'
}

clientdf = result.reset_index().groupby('ClientID').agg(
                AssistanceID = pd.NamedAgg(column='AssistanceID', aggfunc=max)
)

clientdf = (clientdf
                .reset_index()
                .set_index(['ClientID', 'AssistanceID'])
                .join(result)
                .reset_index()
                .drop(columns=['MenClothes', 'MQuant', 'WomenClothes', 
                'WQuant', 'ChildrenClothes', 'CQuant','Soap', 'ShampooConditioner', 
                'Lotion', 'Razor', 'Toothpaste', 'Toothbrush', 'Other', 'BoyAge', 
                'GirlAge', 'Furniture', 'Household', 'Director', 'New', 'Date'])
                .rename(columns={'AssistanceID':'LastAssistanceID'})
)

clientdf['Warning'] = clientdf.ClientID.map(warningmap)
#clientdf.to_json('Clients.json', orient='records')

In [180]:
transaction = assistancedf
transaction = (transaction
                .drop(columns=['BoyAge', 'GirlAge', 'New', 'Date', 'ClientID'])
                .fillna('')
                .assign(Other=lambda df: df['Other'] + df['Furniture'] + df['Household'])
                .drop(columns=['Furniture', 'Household'])
)
transaction = pd.melt(transaction, id_vars=['AssistanceID'], value_vars=['MQuant', 'WQuant', 'CQuant', 'Soap',
       'ShampooConditioner', 'Lotion', 'Razor', 'Toothpaste', 'Toothbrush',
       'Other', 'Director'])
transaction = (transaction
                .loc[(transaction['value'] != 0.0) & (transaction['value'].notna()) & (transaction['value'] != '')]
                .assign(note=lambda df: df['value'])
                .reset_index(drop=True)
                .assign(RequestQuantity=1)
                .rename(columns={'variable':'Item', 'value':'ReceivedQuantity', 'note':'Note'})
)
transaction.loc[transaction['Item'].isin({'MQuant', 'WQuant', 'CQuant', 'Soap',
       'ShampooConditioner', 'Lotion', 'Razor', 'Toothpaste', 'Toothbrush'}), 'Note'] = ''
transaction.loc[transaction['Item'].isin({'MQuant', 'WQuant', 'CQuant'}), 'RequestQuantity'] = 15
transaction.loc[transaction['Item'].isin({'Other', 'Director'}), 'ReceivedQuantity'] = 1

requests = transaction[['AssistanceID', 'Item', 'RequestQuantity', 'Note']]
receipts = transaction[['AssistanceID', 'Item', 'ReceivedQuantity', 'Note']]

#requests.to_json('Requests.json', orient='records')
#receipts.to_json('Receipts.json', orient='records')

In [187]:
assistancedf = (assistancedf
                .drop(columns=['MenClothes', 'MQuant', 'WomenClothes', 
                'WQuant', 'ChildrenClothes', 'CQuant', 'Soap', 'ShampooConditioner', 
                'Lotion', 'Razor', 'Toothpaste', 'Toothbrush', 'Other', 'BoyAge', 
                'GirlAge', 'Furniture', 'Household', 'Director', 'New'])
                .assign(LastBackPackDate=lambda df: df['Date'])
                .assign(LastSleepingBagDate=lambda df: df['Date'])
)
#assistancedf.to_json('Assistances.json', orient='records')